In [102]:
!pip3 install papermill --quiet
!pip install papermill --quiet


error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

In [103]:
import papermill as pm
import os
import glob
import itertools
import pandas as pd


# Change the dataset name and path to your data file here

In [104]:
dataset_name = 'Example_processed_data_020525_after_25000'
dataset_path = f'../data/ICP-MS/{dataset_name}.csv'
notebook_path = 'OutlierRemoval_Final.ipynb'

In [105]:
df = pd.read_csv(dataset_path)

# Change the concentrations here

In [106]:
concentrations = [1,2,3,4,5]

In [107]:
import logging

response_index = None

# set the response data
if not df.columns.empty:
    cols_to_drop = []
    if 'Time' in df.columns:
        cols_to_drop.append('Time')
    if 'Replicate' in df.columns:
        cols_to_drop.append('Replicate')
    if 'Reading' in df.columns:
        cols_to_drop.append('Reading')

    responses = df.drop(columns=cols_to_drop)
    if responses.empty and not df.empty: # Check if all columns were dropped, and df wasn't empty initially
        raise ValueError("All columns were identified as non-response columns. No response data left.")
else:
    raise ValueError("DataFrame has no columns.")

logging.info(f"Available responses: {responses.columns.tolist()}")

os.makedirs(f'../output/{dataset_name}', exist_ok=True)

In [108]:
for response in responses:
    if not os.path.exists(f'../output/{dataset_name}'):
        logging.info(f'../output/{dataset_name} does not exist, creating...')
        os.makedirs(f'../output/{dataset_name}', exist_ok=True)
    else:
        logging.info(f'../output/{dataset_name} already exists.')

    if not os.path.exists(f'../output/{dataset_name}/nb'):
        logging.info(f'../output/{dataset_name}/nb does not exist, creating...')
        os.makedirs(f'../output/{dataset_name}/nb', exist_ok=True)
    else:
        logging.info(f'../output/{dataset_name}/nb already exists.')


    pm.execute_notebook(
                notebook_path,
                f'../output/{dataset_name}/nb/OutlierRemoval-{dataset_name}.ipynb',
                parameters=dict(
                    concentrations=concentrations,
                    dataset_name=dataset_name,
                    path_to_data=dataset_path,
                    response=response,
                    output_path=f'../output/{dataset_name}',)
                    )
    



Executing: 100%|██████████| 24/24 [00:10<00:00,  2.23cell/s]
